In [1]:
from core_pro.ultilities import make_sync_folder
from datasets import Dataset, DatasetDict
import polars as pl
from sklearn.model_selection import train_test_split
from rich import print
from pattern import pattern, label_list
from tqdm.auto import tqdm
from transformers import Trainer
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    AutoModelForTokenClassification,
    pipeline
)
from transformers import EvalPrediction
from pattern import label2id, label_list, id2label
import numpy as np
import evaluate

path = make_sync_folder('nlp/ner')

In [2]:
file = path / 'raw/tag_ner_bio.parquet'
df = pl.read_parquet(file)
df.head()

item_id,item_name,attribute,tokens,ner_tag,ner_id
i64,str,list[struct[2]],list[str],list[str],list[i64]
10204199660,"""Ốp lưng kính cường lực Huawei …","[{""Huawei"",""Brand""}]","[""Ốp"", ""lưng"", … ""mê""]","[""O"", ""O"", … ""O""]","[0, 0, … 0]"
25069093651,"""Hộp Quà Sữa Tắm & Dưỡng Thể …","[{""300g"",""SKU""}]","[""Hộp"", ""Quà"", … ""300gr""]","[""O"", ""O"", … ""B-SKU""]","[0, 0, … 2]"
17955923825,"""Bình giữ nhiệt cao cấp 880ml S…","[{""Bình giữ nhiệt"",""SKU""}]","[""Bình"", ""giữ"", … ""Trắng""]","[""B-SKU"", ""I-SKU"", … ""O""]","[2, 12, … 0]"
18350256452,"""Hạt Giống Hoa Mãn Đình Hồng Đơ…","[{""Đơn"",""Type""}]","[""Hạt"", ""Giống"", … ""Gam)""]","[""O"", ""O"", … ""O""]","[0, 0, … 0]"
5551471172,"""Bàn phím cơ Ajazz chính hãng ♥…","[{""Bàn phím cơ"",""Type""}]","[""Bàn"", ""phím"", … ""Rainbow""]","[""B-Type"", ""I-Type"", … ""O""]","[8, 18, … 0]"


In [3]:
ds = Dataset.from_polars(df)
train = ds.train_test_split(test_size=0.3, seed=42)
test_valid = train['test'].train_test_split(test_size=0.5, seed=42)
dataset = DatasetDict({
    'train': train['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']}
)
dataset

DatasetDict({
    train: Dataset({
        features: ['item_id', 'item_name', 'attribute', 'tokens', 'ner_tag', 'ner_id'],
        num_rows: 1224507
    })
    test: Dataset({
        features: ['item_id', 'item_name', 'attribute', 'tokens', 'ner_tag', 'ner_id'],
        num_rows: 262395
    })
    valid: Dataset({
        features: ['item_id', 'item_name', 'attribute', 'tokens', 'ner_tag', 'ner_id'],
        num_rows: 262394
    })
})

In [4]:
print(dataset['train'][0])

{
    'item_id': 10365314432,
    'item_name': 'Áo thun loang màu tay lỡ dáng rộng - Hoa cúc loang màu',
    'attribute': [{'attribute_value': 'Hoa', 'attribute_name_patterns': 'Style'}],
    'tokens': ['Áo', 'thun', 'loang', 'màu', 'tay', 'lỡ', 'dáng', 'rộng', '-', 'Hoa', 'cúc', 'loang', 'màu'],
    'ner_tag': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Style', 'O', 'O', 'O'],
    'ner_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]
}

In [5]:
pretrain = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrain)

In [6]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True, max_length=50, truncation=True, padding='max_length')
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(example)
print(tokens)

{
    'item_id': 10365314432,
    'item_name': 'Áo thun loang màu tay lỡ dáng rộng - Hoa cúc loang màu',
    'attribute': [{'attribute_value': 'Hoa', 'attribute_name_patterns': 'Style'}],
    'tokens': ['Áo', 'thun', 'loang', 'màu', 'tay', 'lỡ', 'dáng', 'rộng', '-', 'Hoa', 'cúc', 'loang', 'màu'],
    'ner_tag': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Style', 'O', 'O', 'O'],
    'ner_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]
}

[
    '[CLS]',
    'ao',
    'th',
    '##un',
    'loan',
    '##g',
    'ma',
    '##u',
    'tay',
    'lo',
    'dan',
    '##g',
    'ron',
    '##g',
    '-',
    'ho',
    '##a',
    'cu',
    '##c',
    'loan',
    '##g',
    'ma',
    '##u',
    '[SEP]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]',
    '[PAD]'
]

In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True, max_length=50, truncation=True, padding='max_length')

    labels = []
    for i, label in enumerate(examples[f"ner_id"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_ds = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1224507 [00:00<?, ? examples/s]

Map:   0%|          | 0/262395 [00:00<?, ? examples/s]

Map:   0%|          | 0/262394 [00:00<?, ? examples/s]

In [8]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [9]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="pt")

model = AutoModelForTokenClassification.from_pretrained(
    pretrain, 
    num_labels=len(id2label), 
    id2label=id2label, 
    label2id=label2id
)
log_step = 5000
folder = str(path / f'model/{pretrain}')
training_args = TrainingArguments(
    output_dir=folder,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    weight_decay=0.001,
    learning_rate=1e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=16,
    fp16=True,
    logging_strategy='steps',
    save_strategy='steps',
    eval_strategy='steps',
    save_steps=log_step,
    eval_steps=log_step,
    logging_steps=log_step,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to="none",
    num_train_epochs=3,
    optim='adafactor',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
5000,0.137300,0.048575,0.872767,0.899925,0.886138,0.981873
10000,0.046100,0.040670,0.899558,0.904225,0.901885,0.984639
15000,0.038300,0.036951,0.904720,0.918522,0.911569,0.985900
20000,0.033900,0.034930,0.909666,0.926583,0.918047,0.986820
25000,0.027900,0.033410,0.918580,0.924437,0.921499,0.987410


***** train metrics *****
  epoch                    =        3.0
  total_flos               = 43666822GF
  train_loss               =     0.0529
  train_runtime            = 0:24:01.96
  train_samples_per_second =   2547.589
  train_steps_per_second   =     19.904
